In [76]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [77]:

# Fetch DAX data
ticker_symbol = "^GDAXI"
dax_data = yf.download(ticker_symbol, start="1900-01-01", end="2025-01-13")

# Convert to a Pandas DataFrame
dax_df = pd.DataFrame(dax_data)

# Bundestagswahl data
elections_data = [
    ("1949-08-14", "CDU/CSU, FDP, DP"),
    ("1953-09-06", "CDU/CSU, FDP, DP, GB/BHE"),
    ("1957-09-15", "CDU/CSU, DP"),
    ("1961-09-17", "CDU/CSU, FDP"),
    ("1965-09-19", "CDU/CSU, SPD"),
    ("1969-09-28", "SPD, FDP"),
    ("1972-11-19", "SPD, FDP"),
    ("1976-10-03", "SPD, FDP"),
    ("1980-10-05", "SPD, FDP"),
    ("1983-03-06", "CDU/CSU, FDP"),
    ("1987-01-25", "CDU/CSU, FDP"),
    ("1990-12-02", "CDU/CSU, FDP"),
    ("1994-10-16", "CDU/CSU, FDP"),
    ("1998-09-27", "SPD, Greens"),
    ("2002-09-22", "SPD, Greens"),
    ("2005-09-18", "CDU/CSU, SPD"),
    ("2009-09-27", "CDU/CSU, FDP"),
    ("2013-09-22", "CDU/CSU, SPD"),
    ("2017-09-24", "CDU/CSU, SPD"),
    ("2021-09-26", "SPD, Greens, FDP")
]

[*********************100%***********************]  1 of 1 completed


In [78]:
def calculate_performance(date):
    election_date = datetime.strptime(date, "%Y-%m-%d")
    start_date = election_date - timedelta(weeks=12)
    end_date = election_date + timedelta(weeks=12)
    
    # Ensure we have data in the specified range
    if start_date < dax_df.index[0]:
        return None, None
    
    # Filter the DAX data for the relevant period
    period_data = dax_df[(dax_df.index >= start_date) & (dax_df.index <= end_date)]
    
    if not period_data.empty:
        start_price = period_data['Close'].iloc[0]
        
        # Find the closest date to the election date within the period
        nearest_date = period_data.index[np.abs(period_data.index - election_date).argmin()]
        mid_price = period_data['Close'].loc[nearest_date]
        
        end_price = period_data['Close'].iloc[-1]
        
        before_performance = ((mid_price / start_price) - 1) * 100
        after_performance = ((end_price / mid_price) - 1) * 100
        
        return round(before_performance, 2), round(after_performance, 2)
    
    return None, None

# Calculate performance for each election and store results
performances = []
for date, parties in elections_data:
    before, after = calculate_performance(date)
    
    # Append results ensuring numerical values are stored directly
    performances.append({
        'Election Date': date,
        'Governing Parties': parties,
        'Performance Before (%)': before,
        'Performance After (%)': after
    })

# Create DataFrame with election results and DAX performance
election_performance_df = pd.DataFrame(performances)

# Clean up performance columns to ensure they are numeric and handle NaN values properly
election_performance_df['Performance Before (%)'] = election_performance_df['Performance Before (%)'].astype(float)
election_performance_df['Performance After (%)'] = election_performance_df['Performance After (%)'].astype(float)

print(election_performance_df.tail(10))

   Election Date Governing Parties  Performance Before (%)  \
10    1987-01-25      CDU/CSU, FDP                     NaN   
11    1990-12-02      CDU/CSU, FDP                   -8.55   
12    1994-10-16      CDU/CSU, FDP                   -2.27   
13    1998-09-27       SPD, Greens                  -21.28   
14    2002-09-22       SPD, Greens                  -33.26   
15    2005-09-18      CDU/CSU, SPD                    8.89   
16    2009-09-27      CDU/CSU, FDP                   23.31   
17    2013-09-22      CDU/CSU, SPD                    8.16   
18    2017-09-24      CDU/CSU, SPD                    0.96   
19    2021-09-26  SPD, Greens, FDP                   -0.56   

    Performance After (%)  
10                    NaN  
11                   7.39  
12                  -1.78  
13                  -0.23  
14                   5.59  
15                   7.23  
16                   1.65  
17                   4.30  
18                   4.04  
19                  -0.27  


C:\Users\softw\AppData\Local\Temp\ipykernel_43160\3967435015.py:46: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  election_performance_df['Performance Before (%)'] = election_performance_df['Performance Before (%)'].astype(float)
C:\Users\softw\AppData\Local\Temp\ipykernel_43160\3967435015.py:47: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  election_performance_df['Performance After (%)'] = election_performance_df['Performance After (%)'].astype(float)


In [79]:
# Save the election performance data to a CSV file
election_performance_df.to_csv("data/election_dax_performance.csv", index=False)